## Proceso 4: Clasificación de sentimientos múltiples, como: alegría, ira, culpa, etc.

In [1]:
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction import DictVectorizer

import sys
sys.path.append(sys.path[0] + '/..')

from comun import archivo, cadena
from preprocesamiento import tokenizacion
from aprendizajeAutomatico import entrenamiento, prediccion
from sentimociones.sentimientos.sentimientoMultiple import JOY, FEAR, ANGER, SADNESS, DISGUST, SHAME, GUILT

## Lee dataset con texto en español clasificado con sentimientos múltiples

In [2]:
dataset_sentimientos_es = "sentimientoMultiple_en.txt"
data = archivo.leer_datos(dataset_sentimientos_es)
print("Number of instances: {}".format(len(data)))

Number of instances: 7480


### Función para almacenar las etiquetas, que se basarán en emociones como Alegría, Miedo, Ira, etc.:

In [3]:
emotions = [JOY, FEAR, ANGER, SADNESS, DISGUST, SHAME, GUILT]

X_all = []
y_all = []
for label, text in data:
    y_all.append(cadena.convertirEtiqueta(label, emotions))
    X_all.append(tokenizacion.crear_caracteristica(text, nrange=(1, 4)))

### Separación de datos en conjuntos de entrenamiento y prueba

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_all, y_all, test_size = 0.2, random_state = 100)
vectorizer = DictVectorizer(sparse = True)
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

### Entrenamiento con cuatro modelos de aprendizaje automático para la clasificación múltiple de sentimientos:
1. Clasificación con vectores de Soporte 
2. Clasificación con vectores de soporte lineal
3. Clasificación con randon forest
4. Clasificación con árboles de decisión

In [5]:
svc = SVC()
lsvc = LinearSVC(random_state=100)
rforest = RandomForestClassifier(random_state=100)
dtree = DecisionTreeClassifier()

clifs = [svc, lsvc, rforest, dtree]

modelos = {}
for clf in clifs: 
    modelos[clf.__class__.__name__] = entrenamiento.entrenar_probar(clf, X_train, X_test, y_train, y_test)

c:\tools\ides\anaconda\envs\entorno_prueba\lib\site-packages\sklearn\svm\_base.py:985: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


## Resultado de los modelos

In [6]:
for modelo in modelos:
    print(modelo, modelos[modelo][1])

SVC {'Precisión entrenamiento': 0.9114304812834224, 'Precisión pruebas': 0.47927807486631013}
LinearSVC {'Precisión entrenamiento': 0.9989973262032086, 'Precisión pruebas': 0.5621657754010695}
RandomForestClassifier {'Precisión entrenamiento': 0.9989973262032086, 'Precisión pruebas': 0.5508021390374331}
DecisionTreeClassifier {'Precisión entrenamiento': 0.9989973262032086, 'Precisión pruebas': 0.44719251336898397}


## Definición de la etiqueta múltiple de sentimientos con su respectivo emoji

In [7]:
l = [JOY, FEAR, ANGER, SADNESS, DISGUST, SHAME, GUILT]
l.sort()
label_freq = {}
for label, _ in data: 
    label_freq[label] = label_freq.get(label, 0) + 1

# imprime las etiquetas ordenadas
for l in sorted(label_freq, key=label_freq.get, reverse=True):
    print("{:10}({})  {}".format(cadena.convertirEtiqueta(l, emotions), l, label_freq[l]))

emoji_dict = {JOY:"😂", FEAR:"😱", ANGER:"😠", SADNESS:"😢", DISGUST:"😒", SHAME:"😳", GUILT:"😳"}

joy       (1. 0. 0. 0. 0. 0. 0.)  1084
anger     (0. 0. 1. 0. 0. 0. 0.)  1080
sadness   (0. 0. 0. 1. 0. 0. 0.)  1079
fear      (0. 1. 0. 0. 0. 0. 0.)  1078
disgust   (0. 0. 0. 0. 1. 0. 0.)  1057
guilt     (0. 0. 0. 0. 0. 0. 1.)  1057
shame     (0. 0. 0. 0. 0. 1. 0.)  1045


## Obtiene letra de canción preprocesada

In [20]:
from bdd import mongoDB_cliente
from aprendizajeAutomatico import prediccion

cantante = 'The Weeknd'
cancion = 'Blinding Lights'

canciones = mongoDB_cliente.obtenerColeccion('CANCIONES', {'cantante': cantante, 'cancion': cancion})   
letra_preprocesada = canciones[0]['letra_preprocesada']
letra_cancion_pre = ''
for letra in letra_preprocesada:
    letra_cancion_pre = letra_cancion_pre + letra + " "
print(letra_cancion_pre)

blinding lights ingles ive tryna call ive long enough maybe show love maybe im going withdrawals dont even much turn touch baby chorus look around citys cold empty ones around judge cant see clearly youre gone chorus said im blinded lights cant sleep feel touch said im drowning night im like youre one trust im running time cause see sun light sky hit road overdrive baby chorus citys cold empty ones around judge cant see clearly youre gone chorus said im blinded lights cant sleep feel touch said im drowning night im like youre one trust bridge im calling back let know back let know could never say phone say phone never let go time chorus said im blinded lights cant sleep feel touch said im blinded lights cant sleep feel touch weeknd blinding lights 


## Predicción del modelo con vectores de soporte

In [21]:
from aprendizajeAutomatico import entrenamiento, prediccion

modelo = svc
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😠 anger


## Ejecución del modelo vectores de soporte lineal

In [22]:
modelo = lsvc
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😱 fear


## Ejecución del modelo randon forest

In [23]:
modelo = rforest
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😱 fear


# Ejecución del modelo con árboles de decisión

In [24]:
modelo = dtree
res_prediccion, modelos = prediccion.predecirTexto(modelo, letra_cancion_pre, vectorizer, modelos, cancion, cantante)
print('Clasificación de sentimientos con vectores de soporte: ', emoji_dict[res_prediccion], res_prediccion)

Clasificación de sentimientos con vectores de soporte:  😱 fear


## Actualización del resultado en Base de datos

In [25]:
from logica import cancion_logica
from sentimientos import sentimientoMultiple

cancion_logica.actualizarCancionSentimientoMultiple(canciones[0], sentimientoMultiple.obtenerResultadoPredicciones(modelos))

'documento actualizado correctamente'